# Setup a Pipeline and Stream CPU Utilization Data Using Model-driven Telemetry
##### <sup>Authored by Veena Manuel (veedas@cisco.com)</sup>

The use case illustrates how, with the YANG data model, you can stream telemetry data about CPU utilization. Monitoring CPU utilization ensures efficient storage capabilities in your network. This use case also illustrates setting up a pipeline to consume telemetry data that is streamed from the router.

The following table outlines the specifications of this use case:

| Mode            | Transport       | Encoding | 
|-----------------|---------------- | ---------|
| Dial-out        | gRPC            | GPB      |

> In a Dial-out mode, the router dials out to the receiver to establish a subscription-based telemetry session.

The following image represents a topology with 4 routers; router P1 is connected to a collector to stream telemetry data.

![Topology](Telemetry-topology.png)

Telemetry involves the following workflow:
- Define: You define a subscription to stream data from the router to the receiver. To define a subscription,
you create a destination-group and a sensor-group.
- Deploy: The router establishes a subscription-based telemetry session and streams data to the receiver.
You verify subscription deployment on the router.
- Operate: You consume and analyse telemetry data using open-source tools, and take necessary actions
based on the analysis.

To gain an indepth understanding about telemetry streaming modes, see the [Telemetry Configuration Guide](https://www.cisco.com/c/en/us/support/routers/8000-series-routers/products-installation-and-configuration-guides-list.html).

## Configuration steps
* [Bring up emulator session with the base network and configurations](#step1)
* [Setup a Pipeline](#step2)
* [Configure a Telemetry Session Between the Router and the Collector](#step3)
* [Verify the Telemetry configuration](#step4)
* [View the streamed telemetry data](#step5)
* [Operate on Telemetry Data](#step6)

## <a name="step1"></a>Bring up emulator session with the base network and configurations

In order to set up the emulator, you have to execute the cell below by clicking the play button at the top of this notebook. 

In the cell below, the python module **telemetryrx** sets up the python enviroment and the simulated router test bed along with the base configurations of IPv4 addresses, OSPF configurations, MPLS configurations, Loopback interfaces, etc. After the import, the below code block initialises and brings up the emulator using **sim.start API call.

>The bring up can be slow, 10 minutes +. 


In [ ]:
from telemetryrx import *
sim = Vxr()
sim.no_image_copy=True

sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

>At this point, console access to the router is available. Optionally, you can access the simulated router consoles directly from your laptop  through ssh.

In [22]:
print('Consoles can be reached by:')
print('P1:', get_telnet_cmd(sim, 'rp1'), '\nP3:', get_telnet_cmd(sim, 'rp3'), '\nPE1:', get_telnet_cmd(sim, 'rpe1'), '\nPE3:', get_telnet_cmd(sim, 'rpe3'), '\nser1:', get_telnet_cmd(sim, 'ser1'))
print('or better:')
print('P1:', get_ssh_cmd(sim, 'rp1'), '\nP3:', get_ssh_cmd(sim, 'rp3'), '\nPE1:', get_ssh_cmd(sim, 'rpe1'),  '\nPE3:', get_ssh_cmd(sim, 'rpe3'), '\nser1:', get_ssh_cmd(sim, 'ser1')),
print('The password is cisco123')

Consoles can be reached by:
P1: telnet 172.17.0.2 41198 
P3: telnet 172.17.0.2 38183 
PE1: telnet 172.17.0.2 35151 
PE3: telnet 172.17.0.2 44613 
ser1: telnet 172.17.0.2 43499
or better:
P1: ssh cisco@172.17.0.2 -p62385 
P3: ssh cisco@172.17.0.2 -p64666 
PE1: ssh cisco@172.17.0.2 -p60357 
PE3: ssh cisco@172.17.0.2 -p64555 
ser1: ssh cisco@172.17.0.2 -p64842
The password is cisco123


In [23]:
ports = sim.ports()
ser1_ipaddress = str(ports['ser1']['HostAgent'])
ser1_sshport = str(ports['ser1']['xr_redir22'])
p1_ipaddress = str(ports['rp1']['HostAgent'])
p1_sshport = str(ports['rp1']['xr_redir22'])

Install python modules such as, paramiko to ssh to the server and ncclient to talk to NETCONF-enabled devices. Here, router P1.

In [24]:
%%capture coutput

# Installing pre-requisite module
!pip install paramiko
!pip install paramiko-expect
!pip install ncclient

## <a name="step2"></a>Setup the Pipeline

Pipeline is a well-written Golang–based code that consumes IOS XR telemetry streams directly from routers or indirectly from a pub/sub bus (Kafka). In this example, you setup the pipeline to stream data directly from the router. Once collected, the Pipeline performs transformations of the data and forwards the result to the configured consumer.

Pipeline supports different input transport formats from routers:
- TCP
- gRPC
- UDP

Pipeline supports these encoding formats:
- (compact) GPB
- KV-GPB
- JSON

Pipeline supports streaming telemetry data to the following consumers:
- InfluxDB (TSDB)
- Prometheus (TSDB)
- Apache Kafka
- dump-to-file (mostly for diagnostics purposes)

> In this use case, the data will be dumped to a file.

### Create a file to stream telemetry data

Create a file dumpdata.txt to collect data that is streamed from the router. 

In [25]:
import paramiko
from paramiko_expect import SSHClientInteraction

PROMPT = '.*root@.*'

client = paramiko.SSHClient()
# Set SSH key parameters to auto accept unknown hosts
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=ser1_ipaddress, port=ser1_sshport, username='root', password='cisco123', allow_agent=False, look_for_keys=False)

# Check if the file is copied
interact = SSHClientInteraction(client, display=True)
interact.expect(PROMPT)

interact.send('cd /')
interact.expect(PROMPT)

# Create the file to dump data
interact.send('touch dumpdata.txt')
interact.expect(PROMPT)

# Create the file to dump data
interact.send('ls -l')
interact.expect(PROMPT)

Last login: Wed Apr 21 12:10:06 2021
[root@localhost ~]# cd /
[root@localhost /]# touch dumpdata.txt
[root@localhost /]# ls -l
total 16
lrwxrwxrwx.   1 root root    7 Apr 21 05:57 bin -> usr/bin
dr-xr-xr-x.   4 root root 4096 Apr 21 06:05 boot
drwxr-xr-x   19 root root 3100 Apr 21 12:07 dev
-rw-r--r--    1 root root    0 Apr 21 12:15 dumpdata.txt
drwxr-xr-x.  83 root root 8192 Apr 21 12:07 etc
drwxr-xr-x.   2 root root    6 Apr 11  2018 home
lrwxrwxrwx.   1 root root    7 Apr 21 05:57 lib -> usr/lib
lrwxrwxrwx.   1 root root    9 Apr 21 05:57 lib64 -> usr/lib64
drwxr-xr-x.   2 root root    6 Apr 11  2018 media
drwxr-xr-x.   2 root root    6 Apr 11  2018 mnt
drwxr-xr-x.   4 root root   29 Apr 21 06:04 opt
dr-xr-xr-x  119 root root    0 Apr 21 12:06 proc
dr-xr-x---.   4 root root  166 Apr 21 12:10 root
drwxr-xr-x   27 root root  820 Apr 21 12:10 run
lrwxrwxrwx.   1 root root    8 Apr 21 05:57 sbin -> usr/sbin
drwxr-xr-x.   2 root root    6 Apr 11  2018 srv
dr-xr-xr-x   13 root root    0 

0

### Copy the pipeline files to the server

In [26]:
# Using SFTP for the pipeline file-transfers. 
# Pipeline is a large file so this step takes approx 8-10 minutes to copy pipeline file to the server.

transfer = client.open_sftp()
transfer.put("./pipeline_gpb_grpc.conf", "/pipeline_gpb_grpc.conf")
transfer.put("./pipeline", "/pipeline")
transfer.close()

# Check if the file is copied
interact.send('chmod 777 pipeline')
interact.expect(PROMPT)

interact.send('ls -l pipeline*')
interact.expect(PROMPT)

interact.send('ifconfig')
interact.expect(PROMPT)

chmod 777 pipeline
[root@localhost /]# ls -l pipeline*
-rwxrwxrwx 1 root root 86075854 Apr 21 12:15 pipeline
-rw-r--r-- 1 root root      174 Apr 21 12:15 pipeline_gpb_grpc.conf
[root@localhost /]# ifconfig
eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.254.216  netmask 255.255.255.0  broadcast 192.168.254.255
        inet6 fe80::be:c6ff:feb0:f7f  prefixlen 64  scopeid 0x20<link>
        ether 02:be:c6:b0:0f:7f  txqueuelen 1000  (Ethernet)
        RX packets 7211  bytes 86943102 (82.9 MiB)
        RX errors 0  dropped 8  overruns 0  frame 0
        TX packets 8577  bytes 839059 (819.3 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

eth1: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        ether 02:38:3e:3d:fe:f4  txqueuelen 1000  (Ethernet)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 60  bytes 10088 (9.8 KiB)
        TX errors 0  dropped 0 overruns 0  carri

0

### Run the pipeline

In [27]:
# Run the copied file.
interact.send('./pipeline -config pipeline_gpb_grpc.conf')
interact.expect('Wait for.*')

./pipeline -config pipeline_gpb_grpc.conf
Startup pipeline
Load config from [pipeline_gpb_grpc.conf], logging in [pipeline.log]
Wait for ^C to shutdown


0

## <a name="step3"></a>Configure a telemetry session between the router and the collector using Yang Model

To configure telemetry on the router, you will need to setup the following values for a dial-out mode:
- Create a destination group: Create one or more destinations to collect telemetry data from a router. Define a destination-group to contain the details about the destinations. Include the destination address (ipv4 or ipv6), port, transport, and encoding format in the destination-group.

- Create a Sensor-group: Specify the subset of the data that you want to stream from the router using sensor paths. The Sensor path represents the path in the hierarchy of a YANG data model. Create a sensor-group to contain the sensor paths.
> Here, the sensor path is for streaming CPU utilization.

- Create a Subscription: Subscribe to telemetry data that is streamed from a router. A Subscription binds the destination-group with the sensor-group and sets the streaming method. The streaming method can be Cadence-driven or Event-driven telemetry. Cadence-driven telemetry continually streams data (operational statistics and state transitions) at a configured cadence. Whereas, event-driven telemetry optimizes data that is collected at the receiver and streams data only when a state transition occurs.

>Port number range <1-65535>.

> IMPORTANT: To find the IP address of the collector, check the output of the <b>ifconfig</b> command on the Linux server.

In [8]:
from ncclient import manager
from ncclient.xml_ import to_ele
from lxml import etree

telemetry = '''
<config>
<telemetry-model-driven xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-telemetry-model-driven-cfg">
   <destination-groups>
    <destination-group>
     <destination-id>D1</destination-id>
     <ipv4-destinations>
      <ipv4-destination>
       <ipv4-address>192.168.122.104</ipv4-address>
       <destination-port>20030</destination-port>
       <encoding>gpb</encoding>
       <protocol>
        <protocol>grpc</protocol>
        <no-tls/>
       </protocol>
      </ipv4-destination>
     </ipv4-destinations>
    </destination-group>
   </destination-groups>
   <sensor-groups>
    <sensor-group>
     <sensor-group-identifier>SGroup1</sensor-group-identifier>
     <sensor-paths>
      <sensor-path>
       <telemetry-sensor-path>Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization</telemetry-sensor-path>
      </sensor-path>
     </sensor-paths>
    </sensor-group>
   </sensor-groups>
   <enable></enable>
   <subscriptions>
    <subscription>
     <subscription-identifier>Sub1</subscription-identifier>
     <sensor-profiles>
      <sensor-profile>
       <sensorgroupid>SGroup1</sensorgroupid>
       <sample-interval>3000</sample-interval>
      </sensor-profile>
     </sensor-profiles>
     <destination-profiles>
      <destination-profile>
       <destination-id>D1</destination-id>
      </destination-profile>
     </destination-profiles>
    </subscription>
   </subscriptions>
  </telemetry-model-driven>
 </config>
 '''

def connect(host, port, user, password, source):
      conn = manager.connect(host=host,
                             port=port,
                             username=user,
                             password = password,
                             device_params={'name': 'iosxr'},
                             hostkey_verify=False,
                             allow_agent=False,
                             look_for_keys=False
                            )

      rpc_reply = conn.edit_config(config=telemetry)
      conn.commit()
      print(rpc_reply)

connect(p1_ipaddress, p1_sshport, 'cisco', 'cisco123', 'candidate')

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:ddb22e3d-8322-4123-aad3-031eb3dace75" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



Play the below cell so that the notebook can access the telnet consoles of the devices.

In [9]:
console_ports = sim.ports()
loginp1 = telnetlib.Telnet(str(console_ports['rp1']['HostAgent']) , str(console_ports['rp1']['serial0']))
loginser1 = telnetlib.Telnet(str(console_ports['ser1']['HostAgent']) , str(console_ports['ser1']['serial0']))

Run the following command till the <i>Sensor Path State</i> is <b>Resolved</b>. Note that the state changes takes a few seconds. After the state changes to Resolved, proceed to the next step.

In [13]:
# Show the telemetry configuration applied on the router.
loginp1.write(('''
show tele mod sensor-group SGroup1 internal
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show tele mod sensor-group SGroup1 internal
Wed Apr 21 12:03:18.875 UTC
  Sensor Group Id:SGroup1
    Sensor Path:        Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
    Sensor Path State:  Resolved
      Sysdb Path:       /oper/wdsysmon_fd/gl/<wdsysmon_fd_oper_CPUUtilization_nodeid>
       Yang Path:       Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization


RP/0/RP0/CPU0:P1#


## <a name="step4"></a>Verify the telemetry configuration

Access the telnet consoles of the router to apply and verify the telemetry configuration.

In [14]:
# Show the telemetry configuration applied on the router.
loginp1.write(('''
show running-config telemetry model-driven
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show running-config telemetry model-driven
Wed Apr 21 12:03:25.132 UTC
telemetry model-driven
 destination-group D1
  address-family ipv4 192.168.122.104 port 20030
   encoding gpb
   protocol grpc no-tls
  !
 !
 sensor-group SGroup1
  sensor-path Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
 !
 subscription Sub1
  sensor-group-id SGroup1 sample-interval 3000
  destination-id D1
 !
!

RP/0/RP0/CPU0:P1#


Verify the deployment of the subscription and the telemetry configuration on the router.

In [15]:
# Show the telemetry configuration applied on the router.
loginp1.write(('''
show telemetry model-driven subscription Sub1
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show telemetry model-driven subscription Sub1
Wed Apr 21 12:03:28.439 UTC
Subscription:  Sub1
-------------
  State:       NA
  Sensor groups:
  Id: SGroup1
    Sample Interval:      3000 ms
    Heartbeat Interval:   NA
    Sensor Path:          Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
    Sensor Path State:    Resolved

  Destination Groups:
  Group Id: D1
    Destination IP:       192.168.122.104
    Destination Port:     20030
    Encoding:             gpb
    Transport:            grpc
    State:                NA
    TLS :                 False

  Collection Groups:
  ------------------
  No active collection groups

RP/0/RP0/CPU0:P1#


## <a name="step5"></a>View the streamed telemetry data

After the telemetry session is established, the router streams data to the receiver to create a data lake. Check the collector console to see the streamed data. Here is the data from the Linx server (collector) console. The collector was configured to steam data to <b>dumpdata.txt file</b>. View its contents from the server console by playing the below cell. Click the stop button in the notebook when you want to stop the cell execution.

In [16]:
loginser1.write(('''
cat /dumpdata.txt
''').encode('ascii'))
line = loginser1.read_until(b'/r/n',1)
print("***** VIEW FROM TELNET CONSOLE OF SERVER *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF SERVER *****

[root@localhost ~]# cat /dumpdata.txt
[root@localhost ~]# 


> Snapshot of the data captured from the server.

![StreamedData](dumpdata.png)

>Press <b>Ctrl + C</b> to stop the pipeline and the data stream if you are directly accessing the server console.

![StopPipeline](stoppipeline.png)

## <a name="step6"></a>Operate on telemetry data

You can start consuming and analyzing telemetry data from the data lake using an open-sourced collection
stack based on your requirement. 
For example, the following tools can be used to analyze the collected telemetry data:
- Pipeline is a lightweight tool used to collect data. You can download Network Telemetry Pipeline from Github. You define how you want the collector to interact with routers and where you want to send the
processed data using pipeline.conf file.
- Telegraph (plugin-driven server agent) and InfluxDB (a time series database (TSDB)) stores telemetry
data, which is retrieved by visualization tools. You can download InfluxDB from Github. You define
what data you want to include into your TSDB using the metrics.json file.
- Grafana is a visualization tool that displays graphs and counters for data streamed from the router.

For more information, see http://xrdocs.io/telemetry/.

#### Clean-up the topology

>Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [28]:
# Send ctrl-C when you want to stop the execution of pipeline.
interact.send("\x03")
interact.expect(PROMPT)

interact.send('exit')
interact.expect('.*')

client.close()


^C

Interrupt, stopping gracefully

 Stopping input 'dial_out_'
 Stopping output 'jsonoutput'
Done
[root@localhost /]# 
[root@localhost /]# exit
logout


In [29]:
# close your telnet session
loginp1.close()
loginser1.close()

In [2]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/xkzr0lnawl)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [3]:
sim.clean()

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/xkzr0lnawl)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


In [4]:
# Clean up sim scratch space
shutil.rmtree(sim_dir)